# **``bag-of-words/PPMI``**

In [ ]:
import numpy as np
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from collections import Counter, defaultdict
import math
import re

In [ ]:
#nltk.download('gutenberg')
#nltk.download('punkt')

In [ ]:
def preprocess_text(text):
    """Попередня обробка тексту: нижній регістр, видалення пунктуації."""
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

def tokenize_text(text):
    """Токенізація тексту на слова."""
    return word_tokenize(preprocess_text(text))

def get_vocabulary(corpus):
    """Створення словника унікальних слів з корпусу."""
    vocabulary = set()
    for doc in corpus:
        words = tokenize_text(doc)
        vocabulary.update(words)
    return list(vocabulary)

## **Task 0:** Реалізація Bag-of-Words для довільного тексту

In [ ]:
def create_bow_vectors(corpus):
    """Створення Bag-of-Words представлення для кожного документа в корпусі."""
    vocabulary = get_vocabulary(corpus)
    vocab_to_idx = {word: i for i, word in enumerate(vocabulary)}
    
    bow_vectors = []
    for doc in corpus:
        vector = np.zeros(len(vocabulary))
        words = tokenize_text(doc)
        word_counts = Counter(words)
        
        for word, count in word_counts.items():
            if word in vocab_to_idx:
                vector[vocab_to_idx[word]] = count
        
        bow_vectors.append(vector)
    
    return bow_vectors, vocabulary, vocab_to_idx


## **Task 1:** Розширення для використання N-грам замість слів

In [ ]:
def create_ngram_bow_vectors(corpus, n=2):
    """Створення Bag-of-Words на основі N-грам."""
    all_ngrams = set()
    
    # Збір усіх N-грам з корпусу
    for doc in corpus:
        words = tokenize_text(doc)
        doc_ngrams = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
        all_ngrams.update(doc_ngrams)
    
    ngram_to_idx = {ngram: i for i, ngram in enumerate(all_ngrams)}
    
    # Створення BoW векторів
    ngram_bow_vectors = []
    for doc in corpus:
        vector = np.zeros(len(all_ngrams))
        words = tokenize_text(doc)
        doc_ngrams = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
        ngram_counts = Counter(doc_ngrams)
        
        for ngram, count in ngram_counts.items():
            if ngram in ngram_to_idx:
                vector[ngram_to_idx[ngram]] = count
        
        ngram_bow_vectors.append(vector)
    
    return ngram_bow_vectors, list(all_ngrams), ngram_to_idx

## **Task 2:** Реалізація PPMI з підрахунком співзустрічань в межах одного параграфа

In [ ]:
def create_paragraph_cooccurrence_matrix(corpus):
    """Створення матриці співзустрічань слів у межах параграфів."""
    # Розбиття корпусу на параграфи і токенізація
    paragraphs = []
    for doc in corpus:
        paras = doc.split('\n\n')  # Розділення на параграфи
        paragraphs.extend([preprocess_text(p) for p in paras if p.strip()])
    
    vocabulary = get_vocabulary(paragraphs)
    vocab_size = len(vocabulary)
    word_to_idx = {word: i for i, word in enumerate(vocabulary)}
    
    # Створення матриці співзустрічань
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size))
    
    for para in paragraphs:
        words = tokenize_text(para)
        word_indices = [word_to_idx[w] for w in words if w in word_to_idx]
        
        # Підрахунок співзустрічань у параграфі
        for i in word_indices:
            for j in word_indices:
                if i != j:  # Виключаємо співзустрічання слова з самим собою
                    cooccurrence_matrix[i, j] += 1
    
    return cooccurrence_matrix, vocabulary, word_to_idx

## **Task 3:** Реалізація PPMI з підрахунком співзустрічань у вікні сусідніх слів

In [ ]:
def create_window_cooccurrence_matrix(corpus, window_size=5):
    """Створення матриці співзустрічань слів у межах вікна."""
    vocabulary = get_vocabulary(corpus)
    vocab_size = len(vocabulary)
    word_to_idx = {word: i for i, word in enumerate(vocabulary)}
    
    # Створення матриці співзустрічань
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size))
    
    for doc in corpus:
        words = tokenize_text(doc)
        word_indices = [word_to_idx[w] for w in words if w in word_to_idx]
        
        # Підрахунок співзустрічань у вікні
        for i, center_word_idx in enumerate(word_indices):
            window_start = max(0, i - window_size)
            window_end = min(len(word_indices), i + window_size + 1)
            
            for j in range(window_start, window_end):
                if i != j:  # Виключаємо співзустрічання слова з самим собою
                    context_word_idx = word_indices[j]
                    cooccurrence_matrix[center_word_idx, context_word_idx] += 1
    
    return cooccurrence_matrix, vocabulary, word_to_idx

# Функція для обчислення матриці PPMI на основі матриці співзустрічань
def calculate_ppmi_matrix(cooccurrence_matrix):
    """Обчислення матриці PPMI на основі матриці співзустрічань."""
    # Сума всіх співзустрічань
    total_count = np.sum(cooccurrence_matrix)
    
    # Обчислення маргінальних вірогідностей
    word_marginals = np.sum(cooccurrence_matrix, axis=1) / total_count
    context_marginals = np.sum(cooccurrence_matrix, axis=0) / total_count
    
    # Розрахунок матриці p_ij
    p_ij = cooccurrence_matrix / total_count
    
    # Створення матриці PPMI
    ppmi_matrix = np.zeros_like(cooccurrence_matrix, dtype=float)
    
    for i in range(cooccurrence_matrix.shape[0]):
        for j in range(cooccurrence_matrix.shape[1]):
            if p_ij[i, j] > 0:
                pmi = np.log2(p_ij[i, j] / (word_marginals[i] * context_marginals[j]))
                ppmi_matrix[i, j] = max(0, pmi)  # PPMI = max(PMI, 0)
    
    return ppmi_matrix

## **Task 4:** Вирішення проблеми рідких слів з високими значеннями PMI

In [ ]:
def calculate_smoothed_ppmi_matrix(cooccurrence_matrix, alpha=0.75):
    """Обчислення PPMI з використанням згладжування для рідких слів."""
    # Застосування згладжування до підрахунків співзустрічань
    smoothed_matrix = np.power(cooccurrence_matrix, alpha)
    
    # Розрахунок PPMI з використанням згладжених значень
    return calculate_ppmi_matrix(smoothed_matrix)

In [ ]:
def main():
    # Приклад тексту з корпусу Gutenberg
    sample_text = gutenberg.raw('austen-emma.txt')
    
    # Розділення на документи (для простоти використовуємо параграфи)
    paragraphs = [p for p in sample_text.split('\n\n') if p.strip()]
    corpus = paragraphs[:100]  # Беремо перші 100 параграфів для прикладу
    
    print(f"Корпус містить {len(corpus)} документів")
    
    # Task 0: Створення Bag-of-Words представлення
    bow_vectors, vocabulary, vocab_to_idx = create_bow_vectors(corpus)
    print(f"\nTask 0: Bag-of-Words представлення")
    print(f"Розмір словника: {len(vocabulary)}")
    print(f"Форма BoW векторів: {np.array(bow_vectors).shape}")
    
    # Task 1: Використання біграм замість слів
    bigram_bow_vectors, bigrams, bigram_to_idx = create_ngram_bow_vectors(corpus, n=2)
    print(f"\nTask 1: Bag-of-Words з біграмами")
    print(f"Кількість біграм: {len(bigrams)}")
    print(f"Форма BoW-біграм векторів: {np.array(bigram_bow_vectors).shape}")
    
    # Task 2: Матриця співзустрічань у параграфах та PPMI
    paragraph_cooccurrence_matrix, para_vocab, para_word_to_idx = create_paragraph_cooccurrence_matrix(corpus)
    paragraph_ppmi_matrix = calculate_ppmi_matrix(paragraph_cooccurrence_matrix)
    print(f"\nTask 2: PPMI на основі співзустрічань у параграфах")
    print(f"Форма матриці співзустрічань: {paragraph_cooccurrence_matrix.shape}")
    print(f"Форма матриці PPMI: {paragraph_ppmi_matrix.shape}")
    
    # Task 3: Матриця співзустрічань у вікні і PPMI
    window_cooccurrence_matrix, window_vocab, window_word_to_idx = create_window_cooccurrence_matrix(corpus, window_size=5)
    window_ppmi_matrix = calculate_ppmi_matrix(window_cooccurrence_matrix)
    print(f"\nTask 3: PPMI на основі вікна співзустрічань")
    print(f"Форма матриці співзустрічань з вікном: {window_cooccurrence_matrix.shape}")
    print(f"Форма матриці PPMI з вікном: {window_ppmi_matrix.shape}")
    
    # Task 4: Згладжене PPMI для вирішення проблеми рідких слів
    smoothed_ppmi_matrix = calculate_smoothed_ppmi_matrix(window_cooccurrence_matrix, alpha=0.75)
    print(f"\nTask 4: Згладжене PPMI для рідких слів")
    print(f"Форма згладженої матриці PPMI: {smoothed_ppmi_matrix.shape}")
    
    # Task 5: Порівняння з тезаурусом для декількох слів
    common_words = ['love', 'happy', 'sad', 'good', 'bad']
    words_in_vocab = [w for w in common_words if w in para_word_to_idx]
    
    if words_in_vocab:
        print(f"\nTask 5: Аналіз семантично пов'язаних слів")
        for word in words_in_vocab:
            word_idx = para_word_to_idx[word]
            
            # Отримання найближчих слів за PPMI
            ppmi_scores = window_ppmi_matrix[word_idx]
            top_indices = np.argsort(ppmi_scores)[::-1][:10]  # Топ-10 слів
            
            print(f"\nНайближчі слова до '{word}':")
            for idx in top_indices:
                if ppmi_scores[idx] > 0:
                    print(f"  {window_vocab[idx]}: {ppmi_scores[idx]:.4f}")

if __name__ == "__main__":
    main()

Корпус містить 100 документів

Task 0: Bag-of-Words представлення
Розмір словника: 1706
Форма BoW векторів: (100, 1706)

Task 1: Bag-of-Words з біграмами
Кількість біграм: 6119
Форма BoW-біграм векторів: (100, 6119)

Task 2: PPMI на основі співзустрічань у параграфах
Форма матриці співзустрічань: (1706, 1706)
Форма матриці PPMI: (1706, 1706)

Task 3: PPMI на основі вікна співзустрічань
Форма матриці співзустрічань з вікном: (1706, 1706)
Форма матриці PPMI з вікном: (1706, 1706)

Task 4: Згладжене PPMI для рідких слів
Форма згладженої матриці PPMI: (1706, 1706)

Task 5: Аналіз семантично пов'язаних слів

Найближчі слова до 'love':
  goodness: 8.0818
  surprized: 8.0818
  fell: 8.0818
  cease: 8.0818
  nobody: 6.1943
  wanted: 5.7599
  husband: 5.2745
  churchill: 4.9119
  though: 4.6623
  did: 4.2745

Найближчі слова до 'happy':
  months: 6.8594
  square: 6.8594
  brunswick: 6.8594
  sending: 6.8594
  collect: 6.8594
  animated: 6.8594
  unite: 6.8594
  frequently: 6.8594
  circumstance